The following approach was original inspired by this [tutorial](https://www.youtube.com/watch?v=FB5EdxAGxQg&ab_channel=codebasics).

##**Loading the dataset**


In [ ]:
import torchvision.transforms as transforms
import torchvision
import torch
from random import seed 

seed(40)
torch.manual_seed(40)

# Transformations applied to images as they are loaded by the Pytorch dataloader
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

#where we'll store the data
google_drive_path = './drive/MyDrive/Colab Notebooks/3337_Project'

#download our data 
trainset = torchvision.datasets.CIFAR10(root=google_drive_path, train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root=google_drive_path, train=False, download=True, transform=transform)
#the above code is really just used for DLing all data
#we still need to manually transform it (normalize it)
#and we still need to split it into training and testing data

#don't need these anymore
del trainset 
del testset




  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./drive/MyDrive/Colab Notebooks/3337_Project/cifar-10-python.tar.gz to ./drive/MyDrive/Colab Notebooks/3337_Project
Files already downloaded and verified


##**Defining functions for loading the data**

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

np.random.seed(40)

def unpickle(file): #adapted from https://www.cs.toronto.edu/~kriz/cifar.html
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  datadict = unpickle(filename)
  X = datadict[b'data']
  Y = datadict[b'labels']
  X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
  Y = np.array(Y)
  return X, Y

def load_CIFAR10(ROOT): #adapted from tutorial #this splits the data into training and testing sets
  """ load all of cifar """
  xs = []
  ys = []
  for b in range(1,6):
    f = ROOT + '/data_batch_%d'%(b,)
    X, Y = load_CIFAR_batch(f)
    xs.append(X)
    ys.append(Y)    
  Xtr = np.concatenate(xs)
  Ytr = np.concatenate(ys)
  del X, Y
  Xte, Yte = load_CIFAR_batch(ROOT + '/test_batch')
  return Xtr, Ytr, Xte, Yte

##**Loading the data and preprocessing**


In [ ]:
import pandas as pd
cifar10_dir = './drive/MyDrive/Colab Notebooks/3337_Project/cifar-10-batches-py' #changed

X_train_4D_unNorm, y_train, X_test_4D_unNorm, y_test = load_CIFAR10(cifar10_dir) #splits into training and testing sets

In [ ]:
data = [X_train_4D_unNorm, y_train, X_test_4D_unNorm, y_test]
for each in data:
  print(each.shape)
#still need to reshape and normalize

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [ ]:
#reshape the data
X_train_2D_unNorm = np.reshape(X_train_4D_unNorm, (X_train_4D_unNorm.shape[0],-1))
X_test_2D_unNorm = np.reshape(X_test_4D_unNorm, (X_test_4D_unNorm.shape[0],-1))

#now we don't need the 4D data anymore
del X_train_4D_unNorm
del X_test_4D_unNorm

In [ ]:
#here we show that we successfully transformed the 4D set of images to a 2D set of images
#each column is named column,row,rgb
print(X_train_2D_unNorm.shape)
print(X_test_2D_unNorm.shape)

(50000, 3072)
(10000, 3072)


In [ ]:
#now let's produce our pd.DataFrames

columnLabels = [] #we'll feed this to our pd.DataFrame to label each column for each image
for i in range(32):
  for j in range(32):
    for color in ['r','g','b']:
      columnLabels.append(str(i)+','+str(j)+','+color)

X_train_unNorm = pd.DataFrame(X_train_2D_unNorm, columns=columnLabels)
X_test_unNorm = pd.DataFrame(X_test_2D_unNorm, columns=columnLabels)

#don't need X_train_2D_unNorm or X_test_2D_unNorm anymore
del X_train_2D_unNorm
del X_test_2D_unNorm

###**Normalization**


In [ ]:
#now we need to normalize
#but which normalization should we use? To figure this out let's inspect the data's description using a pd.DataFrame
description = X_train_unNorm.describe()
description

0,0,r        0,0,g  ...       31,31,g       31,31,b
count  50000.000000  50000.00000  ...  50000.000000  50000.000000
mean     130.710740    136.05614  ...    125.850300    114.381860
std       73.412873     72.90798  ...     62.649286     66.077526
min        0.000000      0.00000  ...      0.000000      0.000000
25%       71.000000     77.00000  ...     80.000000     63.000000
50%      128.000000    135.00000  ...    123.000000    106.000000
75%      189.000000    195.00000  ...    168.000000    158.000000
max      255.000000    255.00000  ...    255.000000    255.000000

[8 rows x 3072 columns]

In [ ]:
#let's check for outliers

min = description.transpose()["min"].min()
max = description.transpose()["max"].max()

whiskerLimits = pd.DataFrame([], index=["lowWhiskerLimits","highWhiskerLimits"], columns=columnLabels)
for eachCol in description.columns:
  IQR = description[eachCol]["75%"] - description[eachCol]["25%"]
  whiskerLimits[eachCol]["lowWhiskerLimits"] = description[eachCol]["25%"] - 1.5*IQR
  whiskerLimits[eachCol]["highWhiskerLimits"] = description[eachCol]["75%"] + 1.5*IQR

if min >= whiskerLimits.transpose()["lowWhiskerLimits"].max(): #if the lowest minimum is at or above the highest lowerWhiskerLimit
  print("No lower outliers")
else: print("There are lower outliers")

if max <= whiskerLimits.transpose()["highWhiskerLimits"].min(): #if the highest maximum is at or below the lowest highWhiskerLimit
  print("No upper outliers")
else: print("There are upper outliers")

del description

No lower outliers
No upper outliers


In [ ]:
X_train_unNorm

0,0,r  0,0,g  0,0,b  0,1,r  ...  31,30,b  31,31,r  31,31,g  31,31,b
0       59.0   62.0   63.0   43.0  ...     84.0    123.0     92.0     72.0
1      154.0  177.0  187.0  126.0  ...    142.0    143.0    133.0    144.0
2      255.0  255.0  255.0  253.0  ...     83.0     80.0     86.0     84.0
3       28.0   25.0   10.0   37.0  ...     37.0     72.0     65.0     46.0
4      170.0  180.0  198.0  168.0  ...     78.0     73.0     77.0     80.0
...      ...    ...    ...    ...  ...      ...      ...      ...      ...
49995  145.0  161.0  194.0  151.0  ...     53.0     37.0     39.0     54.0
49996  255.0  245.0  132.0  254.0  ...     81.0    253.0    181.0     76.0
49997   35.0  178.0  235.0   40.0  ...     66.0     12.0     31.0     50.0
49998  189.0  211.0  240.0  186.0  ...    171.0    195.0    190.0    171.0
49999  229.0  229.0  239.0  236.0  ...    162.0    163.0    163.0    161.0

[50000 rows x 3072 columns]

In [ ]:
#since there are no outliers we can use min-max normalization to scale everything to be between 0 and 1
#first we'll normalize our X_train dataFrame
from sklearn.preprocessing import MinMaxScaler
X_train = pd.DataFrame(MinMaxScaler().fit_transform(X_train_unNorm), columns=columnLabels)
#fit_transform uses one argument both to determine the mins and maxs (scaling params) and to do the transform
#to normalize our X_test, we want to use the mins and maxs from the train data, since it's such a bigger sample, so we'll do this normalization in steps, first we'll call fit on the training data, then we'll call transform on the test data
X_test = pd.DataFrame(MinMaxScaler().fit(X_train_unNorm).transform(X_test_unNorm), columns=columnLabels) 

##**Training with a subset of the data**
This utilizes the [sklearn.svm.SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) module. Trains with only a fifth of the training set.

In [ ]:
#now we are ready to make our Support Vector Classifier (a type of support vector machine)
from sklearn.svm import SVC
import time

model = SVC() 

tic=time.time()
model.fit(X_train[0:10000], y_train[0:10000]) #takes 5 minutes
toc = time.time()

print("Training took " + str((toc-tic)/3600) +" hours")

Training took 0.08631280057960086 hours


###**Scoring**


In [ ]:
model.score(X_test,y_test)

0.4716

##**Training with all the data**
This utilizes the [sklearn.svm.SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) module.


In [ ]:
#now we are ready to make our Support Vector Classifier (a type of support vector machine)
from sklearn.svm import SVC
import time

model = SVC() 

tic=time.time()
model.fit(X_train, y_train) #this takes a while (~3.5 hrs)
toc = time.time()

print("Training took " + str((toc-tic)/3600) +" hours")

Training took 3.179871161646313 hours


In [ ]:
#here we save the model
import pickle
pickle_filename = "./drive/MyDrive/Colab Notebooks/3337_Project/SVC_SVM_trained_with_one_batch.pkl"
pickle.dump(model, open(pickle_filename,'wb'))

In [ ]:
%cd ./content/drive/MyDrive/Colab Notebooks/3337_Project/

[Errno 2] No such file or directory: './content/drive/MyDrive/Colab Notebooks/3337_Project/'
/content/drive/MyDrive/Colab Notebooks/3337_Project


In [ ]:
%ls

cifar-10-batches-py/  cifar-10-python.tar.gz  drive/


###**Scoring**


In [ ]:
model.score(X_test,y_test)

0.5437